# short_test.ipynb
Functions to examine rolling correlations between device sensor outputs.

Authors:  
– Jon Clucas, 2017 jon.clucas@childmind.org  
– Arno Klein, 2017  
© 2017, Child Mind Institute, Apache v2.0 License

setup:

In [1]:
%matplotlib inline
from annotate_range import annotation_line
from chart_data import bland_altman_plot, df_devices, linechart, rolling_window, xcorr
from config import short_dir, test_urls
from datetime import datetime, timedelta
from normalize_acc_data import actigraph_acc, geneactiv_acc
from plot_normalized_vector_lengths import baseshift_and_renormalize
from utilities.fetch_data import fetch_check_data, fetch_data, fetch_hash
import json, matplotlib as mpl, numpy as np, os, pandas as pd, matplotlib.pyplot as plt
"""
with open(os.path.join('./line_charts/device_colors.json')) as fp:
    color_key = json.load(fp)
"""
pd.set_option('mode.use_inf_as_null', True)
acc_hashes = {'G1_acc_quicktest': '81649fc8a11bf2e87519fa5bb6c0b1f8',
              'A_acc_quicktest': '083a9ec701d94474558618a3b303b0cc',
              'G2_acc_quicktest': 'a23bb0267c79451026b221434ef025ba'}
if not os.path.exists('./sample_data'):
    os.makedirs('./sample_data')

load normalized data:

In [2]:
df = df_devices([('A', 'ActiGraph'), ('G1', 'GENEActiv'), ('G2', 'GENEActiv')], 'accelerometer quicktest',
     datetime(2017, 4, 28, 15, 30), datetime(2017, 4, 28, 15, 48), acc_hashes)
df.rename(columns={'normalized_vector_length': 'normalized_vector_length_GENEActiv(2)'}, inplace=True)

A_acc_quicktest
Retrieve file from URL: https://osf.io/h3aem/?action=download&version=1
Copy file to cache: sample_data/083a9ec701d94474558618a3b303b0cc/A_acc_quicktest.csv
0
('A', 'ActiGraph')
1
G1_acc_quicktest
Retrieve file from URL: https://osf.io/j5rcu/?action=download&version=1
Copy file to cache: sample_data/81649fc8a11bf2e87519fa5bb6c0b1f8/G1_acc_quicktest.csv
1
('G1', 'GENEActiv')
2
G2_acc_quicktest
Retrieve file from URL: https://osf.io/yxgzt/?action=download&version=1
Copy file to cache: sample_data/a23bb0267c79451026b221434ef025ba/G2_acc_quicktest.csv
2
('G2', 'GENEActiv')
3


UnboundLocalError: local variable 'df' referenced before assignment

In [ ]:
Avalues = df['normalized_vector_length_ActiGraph'].values
G1values = df['normalized_vector_length_GENEActiv'].values
G2values = df['normalized_vector_length_GENEActiv(2)'].values

shiftG1G2 = len(G1values) - np.argmax(np.correlate(G1values, G2values, mode='full'))
shiftG1A = len(G1values) - np.argmax(np.correlate(G1values, Avalues, mode='full'))
shiftG2A = len(G2values) - np.argmax(np.correlate(G2values, Avalues, mode='full'))
shiftGA = np.int(np.mean([shiftG1A, shiftG2A]))
[shiftG1G2, shiftG1A, shiftG2A, shiftGA]

In [ ]:
shift_GA = np.abs(shiftGA)
Avalues_shifted = Avalues[:G1values.shape[0]-shift_GA]
G1values_shifted = G1values[shift_GA:G1values.shape[0]]
G2values_shifted = G2values[shift_GA:G2values.shape[0]]
[np.shape(G1values_shifted), np.shape(G2values_shifted), np.shape(Avalues_shifted)]

In [ ]:
[xcorr(G1values_shifted, G2values_shifted), xcorr(Avalues_shifted, G1values_shifted),
 xcorr(Avalues_shifted, G2values_shifted)]

In [ ]:
shifted_t = [datetime(2017, 4, 28, 15, 30)]
while len(shifted_t) < np.shape(Avalues_shifted)[0]:
    shifted_t.append(shifted_t[-1] + timedelta(seconds=0.0166))
shifted_df = pd.DataFrame({'normalized_vector_length_ActiGraph': Avalues_shifted,
            'normalized_vector_length_GENEActiv': G1values_shifted,
            'normalized_vector_length_GENEActiv(2)': G2values_shifted, 'Timestamp':shifted_t})
shifted_df.set_index('Timestamp', inplace=True)

cut middle portion out when devices were being transferred:

In [ ]:
start1 = datetime(2017,4,28,15,30)
stop1 = datetime(2017,4,28,15,37)
start2 = datetime(2017,4,28,15,40)
stop2 = datetime(2017,4,28,15,48)
cropped_df = shifted_df.loc[(shifted_df.index >= start1) & (shifted_df.index <= stop1) |
                            (shifted_df.index >= start2) & (shifted_df.index <= stop2)].copy()

In [ ]:
linechart(cropped_df, 'ActiGraph vs 2×GENEActiv, shifted, cropped section', line=True, full=True)

In [ ]:
Avalues_cropped = cropped_df['normalized_vector_length_ActiGraph'].values
G1values_cropped = cropped_df['normalized_vector_length_GENEActiv'].values
G2values_cropped = cropped_df['normalized_vector_length_GENEActiv(2)'].values

compute normalized cross-correlations:

In [ ]:
[xcorr(G1values_cropped, G2values_cropped), xcorr(Avalues_cropped, G1values_cropped),
 xcorr(Avalues_cropped, G2values_cropped)]

plot x-second windows:

In [ ]:
start = datetime(2017,4,28,15,30) #shifted_t[0]
stop = datetime(2017,4,28,15,48) #shifted_t[-1]
plot_data = True
while start < stop and plot_data:
    new_start = start + timedelta(seconds=10)
    plot_df = cropped_df.loc[(cropped_df.index >= start) & (cropped_df.index <= new_start)].copy()
    label = '–'.join([start.strftime('%H:%M:%S'), new_start.strftime('%H:%M:%S')])
    plot_data = linechart(plot_df, label, line=True, full=False)
    #print(xcorr(plot_df['normalized_vector_length_GENEActiv'].values,
    #            plot_df['normalized_vector_length_GENEActiv(2)'].values))
    #print(xcorr(plot_df['normalized_vector_length_ActiGraph'].values,
    #            plot_df['normalized_vector_length_GENEActiv'].values))
    #print(xcorr(plot_df['normalized_vector_length_ActiGraph'].values,
    #            plot_df['normalized_vector_length_GENEActiv(2)'].values))
    start = new_start

In [ ]:
np.shape(G1values)

In [ ]:
start = datetime(2017,4,28,15,30) #shifted_t[0]
stop = datetime(2017,4,28,15,48) #shifted_t[-1]
plot_data = True
while start < stop and plot_data:
    new_start = start + timedelta(seconds=10)
    plot_df = cropped_df.loc[(cropped_df.index >= start) & (cropped_df.index <= new_start)].copy()
    label = '–'.join([start.strftime('%H:%M:%S'), new_start.strftime('%H:%M:%S')])
    plot_data = linechart(plot_df, label, line=True, full=True)
    #print(xcorr(plot_df['normalized_vector_length_GENEActiv'].values,
    #            plot_df['normalized_vector_length_GENEActiv(2)'].values))
    #print(xcorr(plot_df['normalized_vector_length_ActiGraph'].values,
    #            plot_df['normalized_vector_length_GENEActiv'].values))
    #print(xcorr(plot_df['normalized_vector_length_ActiGraph'].values,
    #            plot_df['normalized_vector_length_GENEActiv(2)'].values))
    start = new_start